In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import requests
import gmaps
import os
import pprint as pp

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [14]:

# read in weather data for a list of cities
file = '../WeatherPy/output_data/cities_weather.csv'
data = pd.read_csv(file)

df = pd.DataFrame(data)
df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hilo,19.7297,-155.0900,84.27,71,75,12.66,US,1666487042
1,Mar del Plata,-38.0023,-57.5575,50.09,66,0,5.99,AR,1666487197
2,Ushuaia,-54.8000,-68.3000,44.26,81,100,6.91,AR,1666487197
3,Mataura,-46.1927,168.8643,68.52,36,100,10.20,NZ,1666487198
4,Zemio,5.0314,25.1361,69.94,96,100,1.86,CF,1666487198


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
# Access maps with API key
gmaps.configure(api_key=g_key)

In [16]:
# create list of locations by lat/long and weights by humidity
places = [(df.loc[i,'Lat'],df.loc[i,'Lng']) for i in df.index]
humidity = [df.loc[i,'Humidity'] for i in df.index]

# set map format
figure_layout = {
    'width': '700px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(map_type='HYBRID',layout=figure_layout)

# apply heatmap layer to map
heat_layer = gmaps.heatmap_layer(places,
                                 dissipating=False,
                                 point_radius=5)
fig.add_layer(heat_layer)

# show map
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='700px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
# filter list of cities by desired weather conditions
hotel_df = df.loc[
    (df['Max Temp'] < 80) & 
    (df['Max Temp'] > 70) & 
    (df['Wind Speed'] < 10) &
    (df['Cloudiness'] == 0)
]

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
50,Sakakah,29.9697,40.2064,72.07,37,0,7.02,SA,1666487211
226,Carnarvon,-24.8667,113.6333,71.40,67,0,8.34,AU,1666487263
278,Saint-Pierre,-21.3393,55.4781,70.92,68,0,4.61,RE,1666487278
380,Três Lagoas,-20.7511,-51.6783,72.14,66,0,6.24,BR,1666487096
384,Gouré,13.9835,10.2704,79.14,18,0,9.44,NE,1666487310
496,Saddiqabad,28.3062,70.1307,71.73,37,0,2.42,PK,1666487342
517,Sur,22.5667,59.5289,76.77,57,0,6.40,OM,1666487349
553,Nizwá,22.9333,57.5333,70.11,44,0,0.92,OM,1666487360
583,Russellville,35.2784,-93.1338,74.98,60,0,3.44,US,1666487153


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
# API query
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    'key':g_key,
    'radius': 50000,
    'types': 'lodging',
}

# iterate through list of cities and log hotel info
for i in hotel_df.index:
    lat = hotel_df.loc[i,'Lat']
    lng = hotel_df.loc[i,'Lng']
    
    # set query location by city lat and lng
    params['location'] = f'{lat},{lng}'
    
    # run query
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    # try/except syntax to catch errors when there are no results
    try: 
        first_result = results[0]
    except(IndexError):
        print('No results found')
    
    # store hotel name if available    
    hotel_df.loc[i, 'Hotel Name'] = first_result.get('name')

c:\Users\parkm\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
c:\Users\parkm\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [19]:
# show results
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
50,Sakakah,29.9697,40.2064,72.07,37,0,7.02,SA,1666487211,Fakhamat Al Orjoana Apartment
226,Carnarvon,-24.8667,113.6333,71.40,67,0,8.34,AU,1666487263,Hospitality Carnarvon
278,Saint-Pierre,-21.3393,55.4781,70.92,68,0,4.61,RE,1666487278,Alize Plage
380,Três Lagoas,-20.7511,-51.6783,72.14,66,0,6.24,BR,1666487096,Druds Express Hotel
384,Gouré,13.9835,10.2704,79.14,18,0,9.44,NE,1666487310,CSI Kellé
496,Saddiqabad,28.3062,70.1307,71.73,37,0,2.42,PK,1666487342,MIDWAY HOTEL & RESTAURANT
517,Sur,22.5667,59.5289,76.77,57,0,6.40,OM,1666487349,Sur Plaza Hotel
553,Nizwá,22.9333,57.5333,70.11,44,0,0.92,OM,1666487360,Golden Tulip Nizwa
583,Russellville,35.2784,-93.1338,74.98,60,0,3.44,US,1666487153,Hampton Inn Russellville


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [23]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='700px'))